In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
import quandl

sns.set()

In [5]:
#quandl.ApiConfig.api_key = "ByAzesCp4TpPvjqYi4ay" #Pour faire plus de 50 requetes par jours
#start = datetime(2016,1,1)
#end = datetime(2017,1,1)
#s = "CBS"

In [6]:
#logo_all=['AAPL','ABT','ACN','ATVI','ADBE','AMD','GOOGL','GOOG','AMZN','AAL','BLK','CBS','MSFT','FB']
#logo=logo_all[1:]

logo_all=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\logo_all.csv")['Logo'].tolist()
logo=logo_all.copy()
logo.remove('AAPL')

logo_LSTM=['AAPL_LSTM']
for l in logo:
    logo_LSTM.append(l+'_LSTM')


In [7]:
#df_companies=pd.DataFrame()
#df_companies['logo']=logo_all
#df_companies['logo_LSTM']=logo_LSTM
#df_companies.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Liste_entreprise.csv")

In [8]:
#####################Price

#df_all=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total_std.csv",index_col='Date') #teste avec que une entreprise

#Volatility sur 25 jours

df_all=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_std25_total.csv",index_col='Date') #teste avec que une entreprise
df_all=df_all[35:]
#Log Return

#df_all=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_logreturn_total.csv",index_col='Date') #teste avec que une entreprise


df=df_all[:-3] #Les data - les 3 à prédire !
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

df_all.tail(1)

,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,AIV,AMAT,APTV,ADM,ARNC,AJG,AIZ,ATO,T,ADSK
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-08,6.083995,4.910338,1.368321,3.321918,5.761588,2.513616,1.90827,2.444089,0.284957,19.665678,...,1.296928,1.109303,NaN,4.40094,0.550842,1.507649,2.428563,1.198278,0.54329,4.112433


A tester, en supprimant les 35 premiers l'index n'est pas reset !

In [9]:
df=df.reset_index()

# Définition des fonctions

In [10]:
class Model:
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)

        rnn_cells = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = forget_bias
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * 2 * size_layer)
        )
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )

In [11]:
#Paramatre

num_layers = 1
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.7
future_day = 5

In [12]:
def prediction_lstm(df,date_ori):
    minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
    df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
    df_log = pd.DataFrame(df_log)
    tf.reset_default_graph()
    modelnn = Model(
    0.01, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        init_value = np.zeros((1, num_layers * 2 * size_layer))
        total_loss = 0
        for k in range(0, df_log.shape[0] - 1, timestamp):
            index = min(k + timestamp, df_log.shape[0] -1)
            batch_x = np.expand_dims(
                df_log.iloc[k : index, :].values, axis = 0
            )
            batch_y = df_log.iloc[k + 1 : index + 1, :].values
            last_state, _, loss = sess.run(
                [modelnn.last_state, modelnn.optimizer, modelnn.cost],
                feed_dict = {
                    modelnn.X: batch_x,
                    modelnn.Y: batch_y,
                    modelnn.hidden_layer: init_value,
                },
            )
            init_value = last_state
            total_loss += loss
        total_loss /= df_log.shape[0] // timestamp
        if (i + 1) % 100 == 0:
            print('epoch:', i + 1, 'avg loss:', total_loss)
            
    output_predict = np.zeros((df_log.shape[0] + future_day, df_log.shape[1]))
    output_predict[0] = df_log.iloc[0]
    upper_b = (df_log.shape[0] // timestamp) * timestamp
    init_value = np.zeros((1, num_layers * 2 * size_layer))
    for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
        out_logits, last_state = sess.run(
            [modelnn.logits, modelnn.last_state],
            feed_dict = {
                modelnn.X: np.expand_dims(
                    df_log.iloc[k : k + timestamp], axis = 0
                ),
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        output_predict[k + 1 : k + timestamp + 1] = out_logits
    
    out_logits, last_state = sess.run(
        [modelnn.logits, modelnn.last_state],
        feed_dict = {
            modelnn.X: np.expand_dims(df_log.iloc[upper_b:], axis = 0),
            modelnn.hidden_layer: init_value,
        },
    )
    init_value = last_state
    output_predict[upper_b + 1 : df_log.shape[0] + 1] = out_logits
    df_log.loc[df_log.shape[0]] = out_logits[-1]
    date_ori.append(date_ori[-1] + timedelta(days = 1))

    for i in range(future_day - 1):
        out_logits, last_state = sess.run(
            [modelnn.logits, modelnn.last_state],
            feed_dict = {
                modelnn.X: np.expand_dims(df_log.iloc[-timestamp:], axis = 0),
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        output_predict[df_log.shape[0]] = out_logits[-1]
        df_log.loc[df_log.shape[0]] = out_logits[-1]
        date_ori.append(date_ori[-1] + timedelta(days = 1))
        
    df_log = minmax.inverse_transform(output_predict)
    date_ori = pd.Series(date_ori).dt.strftime(date_format = '%Y-%m-%d').tolist()
    return df_log, date_ori


# Forecasting

In [13]:
'AAPL' in logo_all

True

In [14]:
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

df_LSTM=pd.DataFrame()
for l in logo_all[:]:
    df_log, date_ori2 =prediction_lstm(df[["Date",l]],date_ori)
    df_LSTM[l+'_LSTM']=pd.DataFrame(df_log)[0].tolist()
    
df_LSTM.tail() 

                

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
epoch: 100 avg loss: 0.0015216469914058077
epoch: 200 avg loss: 0.0023488109957390023
epoch: 300 avg loss: 0.0018927190321846865
epoch: 400 avg loss: 0.002285288642221866
epoch: 500 avg loss: 0.0007197567741202779


C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


epoch: 100 avg loss: 0.002122810859107535
epoch: 200 avg loss: 0.0010579335999194115
epoch: 300 avg loss: 0.0024819687559532837
epoch: 400 avg loss: 0.0011411622269262903
epoch: 500 avg loss: 0.0021666839250093816


...

C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


epoch: 100 avg loss: nan
epoch: 200 avg loss: nan
epoch: 300 avg loss: nan
epoch: 400 avg loss: nan
epoch: 500 avg loss: nan


C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


epoch: 100 avg loss: 0.004150590332325297
epoch: 200 avg loss: 0.003788782672257328
epoch: 300 avg loss: 0.0022630513633071373
epoch: 400 avg loss: 0.0003111942957107921
epoch: 500 avg loss: 0.002542437161058555


,MMM_LSTM,ABT_LSTM,ABBV_LSTM,ABMD_LSTM,ACN_LSTM,ATVI_LSTM,ADBE_LSTM,AMD_LSTM,AAP_LSTM,AES_LSTM,...,ANSS_LSTM,ANTM_LSTM,AON_LSTM,AOS_LSTM,APA_LSTM,AIV_LSTM,AAPL_LSTM,AMAT_LSTM,APTV_LSTM,ADM_LSTM
194,4.315480,1.475366,3.338932,5.178884,2.229365,1.978180,2.998095,0.306556,19.451953,0.630900,...,2.310868,4.973227,1.804057,3.259446,2.577447,1.298300,5.065431,1.187498,NaN,4.256220
195,4.582275,1.660137,3.289510,4.801923,2.190262,1.873648,3.533528,0.321808,20.026024,0.587416,...,2.613114,4.823492,1.957515,3.481168,2.479536,1.310285,5.184880,1.176876,NaN,3.689938
196,4.883265,1.925244,3.156833,4.236933,2.167694,1.807244,4.102010,0.352488,19.286654,0.520876,...,3.064241,4.737156,2.297171,3.650515,2.560338,1.336107,5.260967,1.161684,NaN,2.905931
197,5.219643,2.206390,3.037402,3.234049,2.072338,1.807770,4.878557,0.421332,16.709980,0.447555,...,3.078428,4.628940,2.792172,4.003664,2.610692,1.389194,5.270358,1.144776,NaN,2.109708
198,5.487511,2.587518,2.873133,2.475744,2.162069,1.903919,5.600000,0.444213,13.889608,0.376120,...,3.262271,4.480877,3.205117,4.315747,2.660851,1.443448,5.351387,1.136698,NaN,2.247321


In [15]:
#date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
#df_log5, date_ori5 =prediction_lstm(df[["Date",'MMM']],date_ori)
#df_LSTM['MMM'+'_LSTM']=pd.DataFrame(df_log5)[0].tolist()
#


In [16]:


df_LSTM


,MMM_LSTM,ABT_LSTM,ABBV_LSTM,ABMD_LSTM,ACN_LSTM,ATVI_LSTM,ADBE_LSTM,AMD_LSTM,AAP_LSTM,AES_LSTM,...,ANSS_LSTM,ANTM_LSTM,AON_LSTM,AOS_LSTM,APA_LSTM,AIV_LSTM,AAPL_LSTM,AMAT_LSTM,APTV_LSTM,ADM_LSTM
0,2.746678,1.263829,2.977300,10.950496,2.121233,1.582992,3.045472,0.236905,4.414233,0.454450,...,2.496719,8.977930,3.036090,3.588603,3.220512,1.146114,7.914326,0.708415,NaN,0.965052
1,2.604462,1.347925,2.837328,11.502473,2.076994,1.385383,3.458163,0.257399,3.069070,0.372114,...,2.639326,8.816739,3.086694,3.499710,3.023498,1.032093,7.419336,0.761793,NaN,0.450033
2,2.506720,1.532234,2.690357,11.359133,2.217411,1.456448,3.177202,0.243733,4.770209,0.415398,...,2.791107,8.751988,3.047426,3.315575,3.198543,1.145909,7.389762,0.723384,NaN,0.536544
3,2.556544,1.487930,2.586582,11.199201,2.315778,1.513432,3.145749,0.245023,4.023721,0.444935,...,2.588377,8.489985,2.980619,3.277602,3.028910,1.106916,7.051504,0.764903,NaN,0.332982
4,2.653760,1.448560,2.476370,10.926137,2.452366,1.431906,3.164338,0.239634,4.272506,0.446472,...,2.543484,8.544436,2.982077,3.333642,3.034087,1.189285,6.987285,0.843652,NaN,0.359546
5,2.789705,1.466732,2.292356,11.097188,2.597506,1.355233,3.199279,0.235208,4.868787,0.459655,...,2.571283,8.408281,2.895457,3.329290,2.981835,1.173093,6.746178,0.838837,NaN,0.681497
6,2.520596,1.498231,2.372945,10.901910,2.656395,1.284487,3.097516,0.236457,4.157387,0.459785,...,2.600001,8.493305,2.948728,3.217503,3.034510,1.184590,6.607978,0.842367,NaN,0.482274
7,2.456314,1.454131,2.275495,10.609546,2.834765,1.199835,3.044875,0.236943,4.529382,0.476904,...,2.561957,7.885571,2.927686,3.169986,2.956802,1.174159,6.377526,0.881368,NaN,0.472192
8,2.532359,1.444410,2.048051,10.361605,2.871272,1.100978,3.038062,0.234150,4.305710,0.490974,...,2.441464,7.776992,2.874181,3.162521,2.992117,1.175147,6.397264,0.898675,NaN,0.355063
9,2.371970,1.473193,2.097228,9.949480,2.899386,1.052868,2.904028,0.234060,4.820348,0.474304,...,2.348216,7.493142,2.833179,2.988652,2.982576,1.188101,6.143486,0.949476,NaN,0.534804


In [21]:
pd.DataFrame(df_log).tail(3)


,0
196,2.905931
197,2.109708
198,2.247321


In [24]:
df_final=df_all.reset_index().join(df_LSTM)
df_final.tail(3)

,Date,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,...,ANSS_LSTM,ANTM_LSTM,AON_LSTM,AOS_LSTM,APA_LSTM,AIV_LSTM,AAPL_LSTM,AMAT_LSTM,APTV_LSTM,ADM_LSTM
194,2016-01-06,5.210299,4.330293,1.355878,3.360956,5.760224,2.175936,2.036133,2.725390,0.294230,...,2.310868,4.973227,1.804057,3.259446,2.577447,1.298300,5.065431,1.187498,NaN,4.256220
195,2016-01-07,5.682090,4.627899,1.356963,3.351274,5.767142,2.336475,1.967336,2.586573,0.288655,...,2.613114,4.823492,1.957515,3.481168,2.479536,1.310285,5.184880,1.176876,NaN,3.689938
196,2016-01-08,6.083995,4.910338,1.368321,3.321918,5.761588,2.513616,1.908270,2.444089,0.284957,...,3.064241,4.737156,2.297171,3.650515,2.560338,1.336107,5.260967,1.161684,NaN,2.905931


In [25]:
df_pred=df_final[-3:]
df_pred=df_pred.set_index('Date')
df_pred

,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,ANSS_LSTM,ANTM_LSTM,AON_LSTM,AOS_LSTM,APA_LSTM,AIV_LSTM,AAPL_LSTM,AMAT_LSTM,APTV_LSTM,ADM_LSTM
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,5.210299,4.330293,1.355878,3.360956,5.760224,2.175936,2.036133,2.725390,0.294230,19.609404,...,2.310868,4.973227,1.804057,3.259446,2.577447,1.298300,5.065431,1.187498,NaN,4.256220
2016-01-07,5.682090,4.627899,1.356963,3.351274,5.767142,2.336475,1.967336,2.586573,0.288655,19.615478,...,2.613114,4.823492,1.957515,3.481168,2.479536,1.310285,5.184880,1.176876,NaN,3.689938
2016-01-08,6.083995,4.910338,1.368321,3.321918,5.761588,2.513616,1.908270,2.444089,0.284957,19.665678,...,3.064241,4.737156,2.297171,3.650515,2.560338,1.336107,5.260967,1.161684,NaN,2.905931


In [27]:
####        STD la ###########

df_pred.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\LSTM_All_and_Pred_std25_Final.csv")